In [2]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [4]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [5]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [6]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=10,
    alpha=0.3,
)

In [7]:
question = "Luật giao thông quy định như thế nào về tổng số điểm của giấy phép lái xe và quy định về việc cộng điểm?"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : luật giao_thông quy_định : điểm của giấy_phép lái_xe được dùng để quản_lý việc chấp_hành pháp_luật về trật_tự , an_toàn giao_thông đường_bộ của người lái_xe trên hệ_thống cơ_sở dữ_liệu về trật_tự , an_toàn giao_thông đường_bộ , bao_gồm 12 điểm . số điểm trừ mỗi lần vi_phạm tùy thuộc tính_chất , mức_độ của hành_vi vi_phạm pháp_luật về trật_tự , an_toàn giao_thông đường_bộ . dữ_liệu về điểm trừ giấy_phép lái_xe của người vi_phạm sẽ được cập_nhật vào hệ_thống cơ_sở dữ_liệu ngay sau khi quyết_định xử_phạt có hiệu_lực thi_hành và thông_báo cho người bị trừ điểm giấy_phép lái_xe biết .

Metadata: {'category': 'General', 'fine_amount': 'None', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: General\nMức phạt: None\nNội dung vi phạm: Luật giao thông quy định: Điểm của giấy phép lái xe được dùng để quản lý việc chấp hành pháp luật về trật tự, an toàn giao thông đường bộ của người lái xe trên hệ thống cơ

In [8]:
# client.collections.delete_all()

In [9]:
def classify_query(question: str) -> str:
        # Keywords indicating violation/penalty questions
        violation_keywords = [
            "xử phạt", "bị phạt", "bị gì", "xử lý", "nộp phạt", 
            "phạt tiền", "phạt bao nhiêu", "mức phạt", "tước giấy phép",
            "trừ điểm", "phạt như thế nào", "bị phạt gì", "hình thức xử phạt",
            "xử phạt hành chính", "tiền phạt", "phạt hành chính", "bị tịch thu",
            "thu giữ", "tạm giữ"
        ]
        
        question_lower = question.lower()
        
        # Check if any violation keywords are in the question
        for keyword in violation_keywords:
            if keyword in question_lower:
                return "violation_penalty"
        
    
        return "general_knowledge"